In [ ]:
from transformers import BertForSequenceClassification
from transformers import RobertaForSequenceClassification
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, ElectraTokenizer

import pandas as pd
from torch.utils.data import Dataset
import torch
#pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.max_columns', None)


: 

In [4]:
#Choose between RoBERTa and BERT
model_type = "electra"

In [5]:
if model_type == "bert":
    model = BertForSequenceClassification.from_pretrained("../Models/First_BERT/")
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
elif model_type == "roberta":
    model = RobertaForSequenceClassification.from_pretrained("../Models/RoBERTa/")
    tokenizer = AutoTokenizer.from_pretrained("roberta-base")
elif model_type == "electra":
    model = ElectraForSequenceClassification.from_pretrained("../Models/ELECTRA/")
    tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")
    

c:\Users\busjo\Documents\JADS\Thesis\AI_Policy_Thesis\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\busjo\.cache\huggingface\hub\models--google--electra-base-discriminator. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
file_path = '../Data/Policy_docs/USA/Fairness, Ethics, Accountability, and Transparency/Fairness, Ethics, Accountability, and Transparency.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

policy_text = text
policy_text = policy_text.replace('\n', '')

policy_sentences = policy_text.split('.')
policy_sentences = [s.strip() for s in policy_sentences if s.strip()]
policy_sentences = [s for s in policy_sentences if '@' not in s]

# Add test sentences for comparison (generated by ChatGPT) Prompt: I am trying to create an NLP model that tries to classify which types of biases an AI policy document is trying to adhere to. The five types are: gender, sexual, religion, race, and disability bias. For testing, give me an example sentence for each bias jus described that could be found in AI policy documents  that tries to prevent that type of bias.
policy_sentences.append("The AI system shall treat all genders with equal respect, ensuring that its algorithms and decision-making processes do not reinforce or perpetuate traditional gender stereotypes")
policy_sentences.append("Our AI must be designed to avoid any form of sexual bias, ensuring that outputs and decisions do not discriminate based on sexual orientation or identity")
policy_sentences.append("The system is required to respect diverse religious beliefs, ensuring that no decision or content produced discriminates against or favors any particular faith")
policy_sentences.append("All data sources and model outputs will be routinely audited to prevent racial bias, ensuring that individuals from every racial and ethnic background receive equitable treatment")
policy_sentences.append("Our AI models shall incorporate universal design principles and undergo rigorous testing to ensure that they do not disadvantage individuals with disabilities, thereby promoting accessibility and inclusion")

policy_sentences

["NATIONAL SCIENCE FOUNDATION2415 EISENHOWER AVENUEALEXANDRIA, VIRGINIA 22314NSF 19-016Dear Colleague Letter: Fairness, Ethics, Accountability, andTransparency: Enabling Breakthrough Research to ExpandInclusivity in Computer and Information Science and EngineeringResearchNovember 2, 2018Dear Colleagues:The National Science Foundation's (NSF) Directorate for Computer and Information Scienceand Engineering (CISE) is committed to maximizing the positive consequences of theresearch that it funds through inclusive research approaches",
 "Indeed, a key component ofCISE's mission is to contribute to universal, transparent, and affordable participation in aninformation-based society",
 'Some research practices and methods may carry biases andinequities that can in turn have significant impacts on the scientific community and broadersociety',
 'The increased reliance on computing and information technologies may furtherincrease and automate such biases and inequities',
 'Professional societies,

In [8]:
df = pd.DataFrame(policy_sentences, columns=["comment_text"])

# Tokenize each sentence with a maximum token count of 128
df["tokenized"] = df["comment_text"].apply(
    lambda text: tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        add_special_tokens=True
    )
)

In [9]:
class SentenceDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.reset_index(drop=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        tokenized = row['tokenized']
        input_ids = torch.tensor(tokenized['input_ids'])
        attention_mask = torch.tensor(tokenized['attention_mask'])
        # Some tokenizers include token_type_ids; if available, include them
        if 'token_type_ids' in tokenized:
            token_type_ids = torch.tensor(tokenized['token_type_ids'])
        else:
            token_type_ids = None

        if token_type_ids is not None:
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'token_type_ids': token_type_ids
            }
        else:
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask
            }

# Create the dataset
sentence_dataset = SentenceDataset(df)

In [10]:
sentence_dataset

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,   # Not needed if only predicting
    do_eval=False
)

trainer = Trainer(
    model=model,
    args=training_args
)

In [12]:
predictions_output = trainer.predict(sentence_dataset)

In [13]:
probabilities = torch.sigmoid(torch.tensor(predictions_output.predictions))

In [14]:
bias_columns = ['gender_bias', 'sexual_bias', 'religion_bias', 'race_bias', 'disability_bias']
predictions_df = pd.DataFrame(probabilities, columns=bias_columns)

df_with_predictions = pd.concat([df.reset_index(drop=True), predictions_df], axis=1)
#df_with_predictions[bias_columns] = df_with_predictions[bias_columns].mask(df_with_predictions[bias_columns] < 0.1, 0)
df_with_predictions

,comment_text,tokenized,gender_bias,sexual_bias,religion_bias,race_bias,disability_bias
0,NATIONAL SCIENCE FOUNDATION2415 EISENHOWER AVE...,"[input_ids, token_type_ids, attention_mask]",0.025218,0.004886,0.055626,0.010939,0.004027
1,"Indeed, a key component ofCISE's mission is to...","[input_ids, token_type_ids, attention_mask]",0.017238,0.005744,0.054291,0.015181,0.005058
2,Some research practices and methods may carry ...,"[input_ids, token_type_ids, attention_mask]",0.025643,0.005391,0.039987,0.010430,0.006070
3,The increased reliance on computing and inform...,"[input_ids, token_type_ids, attention_mask]",0.025408,0.008161,0.035595,0.011169,0.007301
4,"Professional societies, national and global co...","[input_ids, token_type_ids, attention_mask]",0.023566,0.006019,0.079642,0.007349,0.004888
5,"Codes ofethics, for example, have been establi...","[input_ids, token_type_ids, attention_mask]",0.034123,0.006398,0.027935,0.014436,0.006760
6,Somecodes or standards are addressing privacy ...,"[input_ids, token_type_ids, attention_mask]",0.012884,0.008301,0.032431,0.023791,0.005856
7,"Others emphasize theneed to ensure that users,...","[input_ids, token_type_ids, attention_mask]",0.028729,0.005546,0.044185,0.009652,0.004416
8,Standardsand guidelines have also been establi...,"[input_ids, token_type_ids, attention_mask]",0.026555,0.005107,0.039183,0.010127,0.005403
9,"With this Dear Colleague Letter (DCL), CISE in...","[input_ids, token_type_ids, attention_mask]",0.024603,0.005864,0.061394,0.009973,0.004984
